In [0]:
from google.colab import drive
drive.mount('/content/drive')
basedir = '/content/drive/My Drive/PSDA Group 4/Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFECV

In [0]:
data_path = basedir + 'wdbc.data'

In [0]:
data = pd.read_csv(data_path,header=None)
data #check the rows and columns of this dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [0]:
x = np.zeros((569, 1), dtype=float) #set array to store the serial number
y = np.zeros((569, 30), dtype=float) #set array to store the values
z = np.zeros((569,), dtype=str)# #set array to store the labels

In [0]:
#read the dataset into array
with open(data_path, 'r') as input_file:
    reader = csv.reader(input_file)
    for i, row in enumerate(reader):
        data_name = [float(row[0])] 
        x[i] = data_name
        data_value = [float(datum) for datum in row[2:] ]
        y[i] = data_value  
        z[i] = row[1] #label
#     y = np.concatenate((x, y), axis=1)

In [0]:
#Encode the labels with value 1 and  0
le = preprocessing.LabelEncoder()
le.fit(["M", "B"])  #Label: M(M=malignant) is 1, B(B=benign) is 0.
z = le.transform(z)
# print('label standard:%s' % data_label)
# print('reverse the label:%s' % le.inverse_transform([0, 0, 1]))

In [0]:
#Split the dataset into training set and test set
y_train, y_test, z_train, z_test = train_test_split(y, z, test_size = 0.2, random_state = 1)

#Transformer and Estimator step by step

In [0]:
#=============================================================

Feature scaling

In [0]:
ytrain_transformed = StandardScaler().fit_transform(y_train)
ytest_transformed = StandardScaler().fit_transform(y_test)

Use PCA to reduce the dimensionality

In [0]:
pca = PCA(n_components=2)
y_train_pca = pca.fit_transform(ytrain_transformed)
y_test_pca = pca.fit_transform(ytest_transformed)


Classifier

In [0]:
classifier = LogisticRegression(random_state=1, solver='liblinear')
classifier.fit(y_train_pca, z_train) #z_train is label
print('Test accuracy: %.3f' % classifier.score(y_test_pca, z_test))

Test accuracy: 0.789


In [0]:
#=============================================================

#Use pipeline to pack all steps(Transformer, Estimator)

---



In [0]:
pipeline = Pipeline([
    ('sc', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ('clf', LogisticRegression(random_state=1, solver='liblinear'))
])
pipeline.fit(y_train, z_train)
print('Test accuracy: %.3f' % pipeline.score(y_test, z_test))

Test accuracy: 0.947


# Recursive Feature Elimination

In [0]:
selector = RFECV(LogisticRegression(random_state=1, solver='liblinear'), step=1, cv=5)
selector = selector.fit(y, z)
print(selector.support_)
print(selector.ranking_)

[ True  True  True False  True  True  True  True  True  True  True  True
  True  True False False  True  True  True False  True  True  True  True
  True  True  True  True  True  True]
[1 1 1 5 1 1 1 1 1 1 1 1 1 1 2 3 1 1 1 4 1 1 1 1 1 1 1 1 1 1]


We see that 26 of the 30 features are relevant. The following features (according to their index number) are relevant and hence used.

In [0]:
y_train_rfe = y_train[:,[0,1,2,4,5,6,7,8,9,10,11,12,13,16,17,18,20,21,22,23,24,24,26,27,28,29]]
y_test_rfe = y_test[:,[0,1,2,4,5,6,7,8,9,10,11,12,13,16,17,18,20,21,22,23,24,24,26,27,28,29]]

In [0]:
pipeline = Pipeline([
    ('sc', StandardScaler()),
    ('clf', LogisticRegression(random_state=1, solver='liblinear'))
])
pipeline.fit(y_train_rfe, z_train)
print('Test accuracy: %.3f' % pipeline.score(y_test_rfe, z_test))

Test accuracy: 0.974


With the Feature Elimination instead of the PCA we improve our accuracy score to 97.4%

#Conclusion

In a typical machine learning workflow you will need to apply all these transformations at least twice. Once when training the model and again on any new data you want to predict on. Of course you could write a function to apply them and reuse that but you would still need to run this first and then call the model separately. Scikit-learn pipelines are a tool to simplify this process. They have several key benefits:
*   They make your workflow much easier to read and understand.
*   They enforce the implementation and order of steps in your project.
*   These in turn make your work much more reproducible.

Scikit-learn has built in functions for most of these commonly used transformations in it’s package.



